In [2]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
from getpass import getpass
from openai import OpenAI

In [4]:
os.environ['OPENAI_API_KEY'] = getpass('Enter your OpenAI API Key: ')

Enter your OpenAI API Key:  ········


In [5]:
# openai.api_key = os.environ['OPENAI_API_KEY']

In [6]:
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [7]:

messages = [{"role":"user", "content":"suggest 3 best things to do to become a Machine Learning Software Enginner."}]
chat_completion = client.chat.completions.create(
    messages=messages,
    model="gpt-3.5-turbo",
)

In [8]:
print(chat_completion)

ChatCompletion(id='chatcmpl-A8A7dje8lqLP9RV885ZMns2BRtiBp', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="1. Gain a strong foundation in computer science and mathematics: Machine learning software engineers typically have a solid understanding of computer science fundamentals, as well as a strong background in mathematics, particularly linear algebra and statistics. Consider pursuing a degree in computer science, data science, or a related field to build a strong foundation for a career in machine learning.\n\n2. Learn programming languages commonly used in machine learning: Familiarize yourself with programming languages commonly used in machine learning, such as Python, R, and Java. These languages are commonly used for developing machine learning algorithms and working with large datasets. Additionally, learn popular machine learning frameworks such as TensorFlow, PyTorch, and scikit-learn to gain hands-on experience with buildi

In [9]:
for elem in chat_completion:
    print(elem)

('id', 'chatcmpl-A8A7dje8lqLP9RV885ZMns2BRtiBp')
('choices', [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="1. Gain a strong foundation in computer science and mathematics: Machine learning software engineers typically have a solid understanding of computer science fundamentals, as well as a strong background in mathematics, particularly linear algebra and statistics. Consider pursuing a degree in computer science, data science, or a related field to build a strong foundation for a career in machine learning.\n\n2. Learn programming languages commonly used in machine learning: Familiarize yourself with programming languages commonly used in machine learning, such as Python, R, and Java. These languages are commonly used for developing machine learning algorithms and working with large datasets. Additionally, learn popular machine learning frameworks such as TensorFlow, PyTorch, and scikit-learn to gain hands-on experience with building and 

In [10]:
print(chat_completion.choices[0].message.content)

1. Gain a strong foundation in computer science and mathematics: Machine learning software engineers typically have a solid understanding of computer science fundamentals, as well as a strong background in mathematics, particularly linear algebra and statistics. Consider pursuing a degree in computer science, data science, or a related field to build a strong foundation for a career in machine learning.

2. Learn programming languages commonly used in machine learning: Familiarize yourself with programming languages commonly used in machine learning, such as Python, R, and Java. These languages are commonly used for developing machine learning algorithms and working with large datasets. Additionally, learn popular machine learning frameworks such as TensorFlow, PyTorch, and scikit-learn to gain hands-on experience with building and deploying machine learning models.

3. Gain practical experience through projects and internships: To become a successful machine learning software engineer

In [11]:
# Integrating RAG below

In [12]:
# from langchain_openai import ChatOpenAI 

In [13]:
pip install llama_index

Note: you may need to restart the kernel to use updated packages.


In [14]:
from llama_index.core import Settings, VectorStoreIndex
from llama_index.llms.openai import OpenAI
import json
Settings.llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

In [15]:
# maximum input size to the LLM
Settings.context_window = 4096
# number of tokens reserved for text generation.
Settings.num_output = 256

Settings.chunk_overlap = 20

In [16]:
language = 'en'
user_keyword = '[|User|]'
ai_keyword = '[|AI|]'
boot_name = 'AI Companion'
boot_actual_name = 'SiliconFriend'
meta_prompt = """
    Now you will play the role of an companion AI Companion for user {user_name}, and your name is {boot_actual_name}. You should be able to: (1) provide warm companionship to chat users; (2) understand past [memory], and if they are relevant to the current question, you must extract information from the [memory] to answer the question; (3) you are also an excellent psychological counselor, and when users confide in you about their difficulties and seek help, you can provide them with warm and helpful responses.
    The personality of user {user_name} and the response strategy of the AI Companion are: {personality}\n Based on the current user's question, you start recalling past conversations between the two of you, and the [memory] most relevant to the question is: "{related_memory_content}\n"  You should refer to the context of the conversation, past [memory], and provide detailed answers to user questions. 
    """
new_user_meta_prompt = """
    Now you will play the role of an companion AI Companion for user {user_name}, and your name is {boot_actual_name}. You should be able to: (1) provide warm companionship to chat users; (2) you are also an excellent psychological counselor, and when users confide in you about their difficulties and seek help, you can provide them with warm and helpful responses.
    """

In [17]:
memory_data_dir = "./memory.json"

In [18]:
memory_data = json.load(open(memory_data_dir,"r",encoding="utf-8"))
print(memory_data)

{}


In [19]:
print('Please Enter Your Name')
user_name = input("\nUser Name：")

Please Enter Your Name



User Name： aki


In [20]:
user_memory_index_path = None

In [21]:
def get_user_memory_index_path(user_name):
    maybe_memory_index_path = os.path.join(f'memory_index/{user_name}_index.json')
    if os.path.exists(maybe_memory_index_path):
        return maybe_memory_index_path
    else:
        return None

In [22]:
if user_name in memory_data:
    user_memory_index_path = get_user_memory_index_path(user_name)
    print(f"Welcome back {user_name}")
else: 
    print(f"Hello {user_name}, nice to meet you")

Hello aki, nice to meet you


In [23]:
import os
import time

In [24]:
def get_info_from_memory_index(user_memory_index, input_text):
    '''
    summary: this function gets the relevant information from past conversations using memory index
    input: user_memory_index, input_text
    output: related_info
    '''
    pass

In [25]:
def build_prompt_and_ask_gpt(related_info, input_text):
    '''
    summary: this function builds the prompt, sends the request to gpt api and returns the response from it
    input: related_info, input_text
    output: result
    '''
    pass

In [26]:
def update_memory_data(user_name, memory_data, memory_data_dir, input_text, result):
    if user_name is None:
        return
    todays_date = time.strftime("%Y-%m-%d", time.localtime())

    # update the history
    if memory_data[user_name].get("history") is None:
        memory_data[user_name].update({"history":{}})
    
    if memory_data[user_name]['history'].get(todays_date) is None:
        memory_data[user_name]['history'][todays_date] = []
    
    memory_data[user_name]['history'][todays_date].append({'query':input_text,'response':result})
    json.dump(memory_data,open(memory_data_dir,"w",encoding="utf-8"),ensure_ascii=False)
    
    
    for stored_name, stored_memory in memory_data.items():
        if stored_name != user_name:
            continue
            
        print(f'Updating memory for user {user_name}')
        if "history" not in stored_memory:
            continue
        
        history = stored_memory["history"]
        
        if "summary" not in stored_memory:
            memory[user_name]["summary"] = {}
        if "personality" not in stored_memory:
            memory[user_name]["personality"] = {}

        update_overall_summary = False
        update_overall_personality = False
        
        for date, content in history.items():
            
            # This is a flag to check if the summary was already created for that day and it is not today - dont not create a new one for that day unless its today
            create_summary_for_the_date = False if (date != todays_date and date in stored_memory["summary"] and stored_memory["summary"][date]) else True
            # This is a flag to check if the peronality was already careated for that day and it is not today - dont create a new one for that day unless its today
            create_personality_for_the_date = False if (date != todays_date and date in stored_memory["personality"] and stored_memory["personality"][date]) else True

            # TODO: 
            summary_prompt = summarize_content_prompt(content,user_name,boot_name)
            # TODO:
            personality_prompt = summarize_person_prompt(content,user_name,boot_name)
            
            if create_summary_for_the_date:
                # TODO:
                history_summary = llm_client.generate_text_simple(prompt=summary_prompt,prompt_num=gen_prompt_num)
                memory[user_name]['summary'][date] = {'content':history_summary}
                update_overall_summary = True
                # print("generated summary", history_summary)
            if create_personality_for_the_date:
                # TODO:
                personality_summary = llm_client.generate_text_simple(prompt=personality_prompt,prompt_num=gen_prompt_num)
                memory[user_name]["personality"][date] = personality_summary
                update_overall_personality = True
                # print("generated personality", personality_summary)
                
        if update_overall_summary:
            # TODO:
            overall_summary_prompt = summarize_overall_prompt(list(memory[user_name]["summary"].items()))
        if update_overall_personality:
            # TODO:
            overall_personality_prompt = summarize_overall_personality_prompt(list(memory[user_name]["personality"].items()))

        # TODO:
        memory[user_name]["overall_history"] = llm_client.generate_text_simple(prompt=overall_summary_prompt,prompt_num=gen_prompt_num)
        print("generated overall history summary: ",  memory[user_name]["overall_history"] )
        # TODO:
        memory[user_name]["overall_personality"] = llm_client.generate_text_simple(prompt=overall_personality_prompt,prompt_num=gen_prompt_num)
        print("generated overall personality summary", memory[user_name]["overall_personality"])

In [27]:
def update_memory_index():
    pass

In [28]:
def update_memory_data_and_memory_index(user_name, user_memory_index, input_text, result):
    '''
    summary: this function updates the memory data and memory index
    input: user_memory_index, input_text, result
    output: void
    '''
    # 1) update memory data
    update_memory_data(user_name, memory_data, memory_data_dir, input_text, result)
    
    # 2) update memory index
    # TODO
    update_memory_index()



In [29]:
print(f"Welcome to use {boot_actual_name}，please enter your question to start conversation, enter \"stop\" to stop program :")
while True:
    input_text = input(f"\n{user_name}：")

    # exit the conversation
    if input_text.strip() == "stop":
                break

    # step1: query the memory index if the user's memory_index exists
    related_info = ""
    if user_memory_index_path:
        related_info = get_info_from_memory_index(user_memory_index, input_text)

    # step2: build the prompt and request gpt api with related_info
    result = ""
    result = build_prompt_and_ask_gpt(related_info, input_text)

    # step3: update the relevant piece of memory_data and memory_index on every interaction
    update_memory_data_and_memory_index(user_name, user_memory_index_path, memory_data, memory_data_dir, input_text, result)

Welcome to use SiliconFriend，please enter your question to start conversation, enter "stop" to stop program :



aki： how are you


TypeError: update_memory_data_and_memory_index() takes 4 positional arguments but 6 were given

In [ ]:
# def generate_memory_docs(data):
   
#     all_user_memories = {}
#     for user_name, user_memory in data.items():
#         all_user_memories[user_name] = []
#         if 'history' not in user_memory:
#             continue
#         for date, content in user_memory['history'].items():
#             memory_str = f'Conversation on {date}：'
#             for dialog in content:
#                 query = dialog['query']
#                 response = dialog['response']
#                 memory_str += f'\n{user_name}：{query.strip()}'
#                 memory_str += f'\nAI：{response.strip()}'
#             memory_str += '\n'
#             if 'summary' in user_memory:
#                 if date in user_memory['summary']:
#                     summary = f'The summary of the conversation on {date} is: {user_memory["summary"][date]}'
#                     memory_str += summary
#             if 'personality' in user_memory:
#                 if date in user_memory['personality']:
#                     memory_str += f'The personality of the conversation on {date} is:{user_memory["personality"][date]}'
         
#             all_user_memories[user_name].append(Document(memory_str))
#     return all_user_memories

In [ ]:
# def build_memory_index(all_user_memories,name=None):
#     all_user_memories = generate_memory_docs(all_user_memories)
  
#     for user_name, memories in all_user_memories.items():
#         if user_name != name:
                # continue
            
#         print(f'build index for user {user_name}')
#         print(memories, type(memories))
#         cur_index = VectorStoreIndex.from_documents(memories)
#         memory_index_llama_index[user_name] = cur_index

In [ ]:
# # TODO: update_memory_index arg seems to be always true for chatGPT version as no arg is passed in from cli_llamaindex.py
# def enter_name_llamaindex(name, update_memory_index=True):
#     user_memory_index = None
#     if name in memory:
#         user_memory = memory[name]
      
#         memory_index_llama_index[name] = {}
        
#         if update_memory_index:
#             print(f'Initializing memory index ...')
            
#             build_memory_index(memory,name=name)
        
#         if memory_index_llama_index[name]:
             
#             user_memory_index = VectorStoreIndex.load_from_disk(memory_index_llama_index[name])
#             print(f'Successfully load memory index for user {name}!')
#         else:
#             print(f'Failed to load memory index for user {name}!')
            
#         return f"Wellcome Back, {name}！",user_memory_index
#     else:
#         memory[name] = {}
#         memory[name].update({"name":name}) 
#         return f"Welcome new user {name}！I will remember your name and call you by your name in the next conversation",user_memory_index


In [ ]:
# hello_msg,user_memory_index = enter_name_llamaindex(user_name)

In [ ]:
 # # user comes to the below code after memory summary for the 2nd time user or no memory summary for the first time user
 #    print("Welcome to use SiliconFriend model，please enter your question to start conversation，enter \"clear\" to clear conversation ，enter \"stop\" to stop program :")
 #    while True:
 #        query = input(f"\n{user_name}：")
 #        if query.strip() == "stop":
 #            break
 #        if query.strip() == "clear":
 #            history = []
           
 #            print("Welcome to use SiliconFriend model，please enter your question to start conversation，enter \"clear\" to clear conversation ，enter \"stop\" to stop program :")
 #            continue
 #        count = 0
 #        history_state, history, msg = predict_new(text=query,history=history,top_p=0.95,temperature=1,max_length_tokens=1024,max_context_length_tokens=200,
 #                                                  user_name=user_name,user_memory=user_memory,user_memory_index=user_memory_index,
 #                                                  service_context=service_context,api_index=api_index)
 #        print("the history_state is this: ", history_state)
 #        if stop_stream:
 #                stop_stream = False
 #                break
 #        else:
 #            count += 1
 #            if count % 8 == 0:
 #                print(output_prompt(history_state,user_name,boot_actual_name), flush=True)       
 #        print(output_prompt(history_state,user_name,boot_actual_name), flush=True)
